In [ ]:
## GEOG458
## Xinbo Lu, Wenjin Yu, Jody Tran
## 2/15/2019
## Group Lab 3


In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib as mpl
%matplotlib inline

In [2]:
WA_bg = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/saep_bg10_Copy/saep_bg10.shp")
FIPS_to_CountyNames_gdf = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/WashingtonFIPS_copy.dbf")
WA_bg.head(5)

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,OHU2014,OHU2015,OHU2016,OHU2017,OHU2018,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,277.494,279.680,287.530,287.522,287.758,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,..."
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,524.831,521.228,517.427,516.077,524.645,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,..."
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,246.344,243.965,242.033,241.365,245.491,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,..."
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,208.923,203.761,205.502,204.547,202.854,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ..."
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,285.857,286.251,278.398,278.400,267.983,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,..."


In [3]:
print "Initial table: "
print FIPS_to_CountyNames_gdf.head(5)

Initial table: 
  CountyName FIPSCounty geometry
0      Adams        001     None
1     Asotin        003     None
2     Benton        005     None
3     Chelan        007     None
4    Clallam        009     None


In [4]:
FIPS_to_CountyNames_gdf.rename(columns={'FIPSCounty':'COUNTYFP10'},inplace=True)
FIPS_to_CountyNames_gdf.drop('geometry', axis=1, inplace=True)
print
print "After preparation for merge (join): "
print FIPS_to_CountyNames_gdf.head()


After preparation for merge (join): 
  CountyName COUNTYFP10
0      Adams        001
1     Asotin        003
2     Benton        005
3     Chelan        007
4    Clallam        009


In [5]:
JOINING_KEY = 'COUNTYFP10'
WA_bg_with_CountyName = WA_bg.merge(FIPS_to_CountyNames_gdf, on=JOINING_KEY)
WA_bg_with_CountyName.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,OHU2015,OHU2016,OHU2017,OHU2018,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry,CountyName
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,279.680,287.530,287.522,287.758,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,...",Adams
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,521.228,517.427,516.077,524.645,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,...",Adams
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,243.965,242.033,241.365,245.491,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,...",Adams
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,203.761,205.502,204.547,202.854,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ...",Adams
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,286.251,278.398,278.400,267.983,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,...",Adams


In [ ]:
WA_bg_with_CountyName_wgs84 = WA_bg_with_CountyName.to_crs(epsg=4326)
import os  
for county in FIPS_to_CountyNames_gdf['CountyName']:
    filename_to_output = county+".geojson"
    print "Outputting "+filename_to_output
    try: 
        os.remove(filename_to_output)
    except OSError:
        pass
    (WA_bg_with_CountyName_wgs84[WA_bg_with_CountyName['CountyName'] == county]
     .to_file(filename_to_output,driver="GeoJSON"))

In [6]:
## Print out a ranked list, in descending order, of the ten largest total populations 
## in 2017 of counties in Washington, according to this dataset.

pop2018_series = WA_bg_with_CountyName.groupby(['CountyName']).sum()['POP2018'].rank(ascending=False).head(10)
top10_POP2017_county = pop2018_series.index.tolist()

In [68]:
## Part 3
WA_shp = gpd.read_file("C:/Users/LXB19/Documents/GEOG458/GroupLab3/saep_bg10_Copy/saep_bg10.shp")
WA_shp['Pop_Per_sqML_2018'] = WA_shp.POP2018/WA_shp.ALANDMI + 10**-8
WA_shp['Urban_2018'] = np.nan

In [69]:
WA_shp.loc[WA_shp.Pop_Per_sqML_2018 >= 500, 'Urban_2018'] = 'TRUE'
WA_shp.loc[WA_shp.Pop_Per_sqML_2018 < 500, 'Urban_2018'] = 'FALSE'

In [70]:
UrbanArea_2018 = WA_shp[(WA_shp['Pop_Per_sqML_2018'] >= 500)]
UrbanPopPercent_2018 = round(sum(UrbanArea_2018.POP2018 / sum(WA_shp.POP2018)) * 100, 2)
print "The percentage of the population of the state is urbanized in 2018 is", UrbanPopPercent_2018, "%"

The percentage of the population of the state is urbanized in 2018 is 80.48 %


In [71]:
##Write code that calculates and prints out to the screen what percentage 
##     of the land area of the state is urbanized in the most recent year.
totalArea = WA_shp.ALANDMI.sum()
WA_shp.UrbanAreaPercent_2018 = WA_shp.ALANDMI/totalArea * 100
WA_shp.UrbanAreaPercent_2018.head(5)

0    0.928006
1    0.005679
2    0.004869
3    0.628103
4    0.204606
Name: ALANDMI, dtype: float64

In [72]:
## Add a column to the shapefile's data table for whether block groups have 'urbanized', 
## 'no change in category', or 'deurbanized' over the previous decade (i.e. if most recent year is ’18, 
## then ’08-’18), using the above classification for urban/rural you decided on. 
## Represent it as a categorical change (a string of text), not in terms of some numerical change. 
## Fill in the field for all rows! 

WA_shp_copy = WA_shp[['COUNTYFP10','POP2008', 'POP2009', 'POP2010', 'POP2011', 'POP2012', 
                      'POP2013', 'POP2014', 'POP2015', 'POP2016','POP2017','POP2017', 'POP2018',
                      'Pop_Per_sqML_2018','Urban_2018', 'ALANDMI']].copy()

In [73]:
WA_shp['Pop_Per_sqML_2008'] = WA_shp.POP2008/WA_shp.ALANDMI + 10**-8
WA_shp['Urban_2008'] = np.nan
WA_shp['Urban_2008_DecadeChange']=np.nan

In [74]:
WA_shp.loc[WA_shp.Pop_Per_sqML_2008 >= 500, 'Urban_2008'] = 'TRUE'
WA_shp.loc[WA_shp.Pop_Per_sqML_2008 < 500, 'Urban_2008'] = 'FALSE'

In [90]:
pd.options.mode.chained_assignment = 'warn'
for row in WA_shp.itertuples():
    if (WA_shp.Urban_2018[row] == 'FALSE' and WA_shp.Urban_2008,[row] =='FALSE'):
        WA_shp.loc[WA_shp.Urban_2008_DecadeChange, row] = 'no change in category'
    elif(WA_shp.Urban_2018[row] == 'FALSE' and WA_shp.Urban_2008,[row] =='TRUE'):
        WA_shp.loc[WA_shp.Urban_2008_DecadeChange, row] = 'deurbanized'
    else:
        WA_shp.loc[WA_shp.Urban_2008_DecadeChange, row] = 'urbanized'

ValueError: Can only tuple-index with a MultiIndex

In [81]:
WA_shp.Urban_2008_DecadeChange

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
       ... 
4753    NaN
4754    NaN
4755    NaN
4756    NaN
4757    NaN
4758    NaN
4759    NaN
4760    NaN
4761    NaN
4762    NaN
4763    NaN
4764    NaN
4765    NaN
4766    NaN
4767    NaN
4768    NaN
4769    NaN
4770    NaN
4771    NaN
4772    NaN
4773    NaN
4774    NaN
4775    NaN
4776    NaN
4777    NaN
4778    NaN
4779    NaN
4780    NaN
4781    NaN
4782    NaN
Name: Urban_2008_DecadeChange, Length: 4783, dtype: object

In [84]:
WA_shp.head(5)

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,GEOID10,NAMELSAD10,MTFCC10,FUNCSTAT10,INTPTLON10,INTPTLAT10,...,COHU00_10,PCOHU00_10,COHU10_18,PCOHU10_18,geometry,Pop_Per_sqML_2018,Urban_2018,Pop_Per_sqML_2008,Urban_2008,Urban_2008_DecadeChange
0,53,001,950100,1,530019501001,Block Group 1,G5030,S,-118.398815,47.150809,...,33.751,0.143,17.758,0.066,"POLYGON ((2077217.074095237 640954.0110817049,...",1.270921,FALSE,1.192891,FALSE,NaN
1,53,001,950100,2,530019501002,Block Group 2,G5030,S,-118.351143,47.135101,...,3.398,0.007,-0.355,0.000,"POLYGON ((2165913.051240579 657352.8005365322,...",303.724695,FALSE,308.274775,FALSE,NaN
2,53,001,950100,3,530019501003,Block Group 3,G5030,S,-118.382148,47.138222,...,-26.312,-0.097,-0.509,-0.002,"POLYGON ((2166254.454566129 657810.7137267586,...",171.655748,FALSE,173.292027,FALSE,NaN
3,53,001,950200,1,530019502001,Block Group 1,G5030,S,-118.219484,46.874027,...,-21.255,-0.093,-5.146,-0.025,"POLYGON ((2150159.78406948 554295.8999707697, ...",1.006330,FALSE,1.087854,FALSE,NaN
4,53,001,950200,2,530019502002,Block Group 2,G5030,S,-118.522323,46.968569,...,-1.330,-0.005,-22.017,-0.076,"POLYGON ((2082999.834788324 575018.8611698836,...",5.199997,FALSE,5.331252,FALSE,NaN
